[wifi_data_process.ipynb] instruction

This is our first step code. The input file needed for running this file is [updated_densitymap_hourly_cleaned.csv], which include origin wifi data (foot traffic data)given by client. However this input sheet is too large so we do not put it into github, you should download it from  [https://drive.google.com/drive/folders/11Zu-7h4pp9KWnJbkMMpTo6H_6xsmfzBb] and put into [original_data] folder before you run the code.
[Building and Kitchen List.xlsx] in [original_data] folder is also required as the input file for this code.



In this [wifi_data_process.ipynb], we do 2 Data Filter and Processing steps for original wifi data:

1. We exclude Housing, and Commercial buildings because they're private . For residential buildings we only consider large dorm-style residences for similar reason. We also exclude parking building or floors since people usually do not stay long here. 

2. We simplify our analysis by calculating and focusing on hourly mean and max density counts for each floor. This helps us quickly identify when and where traffic peaks.


After running this code, we can get [updated_wifi_data.csv] in [new_data] folder as result data



In [1]:
import pandas as pd
import numpy as np

# READ CSV FILE
df = pd.read_csv('../original_data/updated_densitymap_hourly_cleaned.csv')
df['building_type'] = df['building_type'].str.strip()

# exclude 'building_type' :  'Parking', 'Housing' 'Commercial' 
filtered_df = df[~df['building_type'].isin([ 'Parking', 'Housing', 'Commercial'])]


# calculate max and mean density cnt based on same building floor and building desc
# cnt refers to the number of people in the space at a given point in time
grouped_values = filtered_df.groupby(['building_floor', 'building_desc'], as_index=False).agg(
    mean_density_cnt=('density_cnt', 'mean'),
    max_density_cnt=('density_cnt', 'max')
)

#only record first column for each same building floor + building description
first_row_flags = filtered_df.groupby(['building_floor', 'building_desc']).cumcount() == 0
representative_rows = filtered_df[first_row_flags]

# add mean and max density into file
final_representative_rows = pd.merge(representative_rows, grouped_values, on=['building_floor', 'building_desc'], how='left')

# delete 'date'  'hour' 'density_cnt''dc_ratio'
final_representative_rows_cleaned = final_representative_rows.drop(columns=['date', 'hour','density_cnt','dc_ratio'], errors='ignore')

# add  'average_dc_ratio' and max dc ratio
final_representative_rows_cleaned['average_dc_ratio'] = final_representative_rows_cleaned['mean_density_cnt'] / final_representative_rows_cleaned['capacity']
 
final_representative_rows_cleaned['max_dc_ratio'] = final_representative_rows_cleaned['max_density_cnt'] / final_representative_rows_cleaned['capacity']

# create processed file
#final_representative_rows_cleaned.to_csv('pre_wifi_update.csv', index=False)

#final_representative_rows_cleaned.head()

now continue processing. for residential type, only remain Large Dormitory-Style type.

this can be done by using Building and Kitchen List.xlsx.

In [2]:
#wifi_df = pd.read_csv('pre_wifi_update.csv')
wifi_df = final_representative_rows_cleaned

# read excel
#building_list_df = pd.read_excel('../original_data/Building and Kitchen List.xlsx')
building_list_df = pd.read_excel('../original_data/Building and Kitchen List.xlsx')

# Identify all housing types labeled "Large Dormitory Style" from the Excel file
large_dormitory_buildings = building_list_df[building_list_df['Residential Building Type'] == 'Large Dormitory-Style']['Building Description'].unique()
# Add the "Residential Building Type" column and set it to an empty string by default
wifi_df['Residential Building Type'] = ''

# Set "Residential Building Type" for rows that meet the criteria
wifi_df.loc[wifi_df['building_type'].str.contains('Residential') & wifi_df['building_desc'].isin(large_dormitory_buildings), 'Residential Building Type'] = 'Large Dormitory-Style'

# Filter data in wifi_update.csv
# Keep all rows for non-residential types, for residential types, only those that belong to the "Large Dormitory Style"
filtered_wifi_df = wifi_df[
    (~wifi_df['building_type'].str.contains('Residential')) |
    (wifi_df['building_type'].str.contains('Residential') & wifi_df['building_desc'].isin(large_dormitory_buildings))
]
output_file_path = '../new_data/updated_wifi_data.csv'

filtered_wifi_df.to_csv(output_file_path, index=False)
filtered_wifi_df.head

<bound method NDFrame.head of      building_floor                  building_desc  latitude  longitude  \
8                 1        771 COMMONWEALTH AVENUE   42.3510   -71.1080   
9                 2        771 COMMONWEALTH AVENUE   42.3510   -71.1080   
10                1        775 COMMONWEALTH AVENUE   42.3509   -71.1090   
11                1              1 UNIVERSITY ROAD   42.3512   -71.1100   
12                1  745 - 755 COMMONWEALTH AVENUE   42.3505   -71.1073   
...             ...                            ...       ...        ...   
1134              3    617-621 COMMONWEALTH AVENUE   42.3497   -71.1013   
1135              p             100 BAY STATE ROAD   42.3498   -71.0979   
1136             81              91 BAY STATE ROAD   42.3503   -71.0970   
1138              A        700 COMMONWEALTH AVENUE   42.3493   -71.1040   
1139              W             140 BAY STATE ROAD   42.3500   -71.0998   

        building_type  capacity  mean_density_cnt  max_density_cnt  \